<a href="https://colab.research.google.com/github/ritesh-chafer/Coronavirus_Enrichment/blob/master/Coronavirus_Dataset_Enrichment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls '/content/drive/My Drive/Colab_Notebooks/twitter.properties'

'/content/drive/My Drive/Colab_Notebooks/twitter.properties'


In [3]:
!pip3 install ConfigParser

In [4]:
import configparser
config = configparser.RawConfigParser()
config.read('//content//drive//My Drive//Colab_Notebooks//twitter.properties')
print(config.sections())

['twitter']


In [5]:
import pandas as pd
import numpy as np
import os
import tweepy as tw

In [6]:
access_token=config.get('twitter', 'accesstoken')
access_tokensecret=config.get('twitter', 'accesstokensecret')
api_key=config.get('twitter', 'apikey')
api_secretkey=config.get('twitter', 'apisecretkey')
map_api_key = config.get('twitter', 'googleapikey')

In [7]:
import pandas as pd

In [8]:
corona_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [9]:
corona_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,1/12/22,1/13/22,1/14/22,1/15/22,1/16/22,1/17/22,1/18/22,1/19/22,1/20/22,1/21/22,1/22/22,1/23/22,1/24/22,1/25/22,1/26/22,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,5,...,158511,158602,158639,158678,158717,158826,158974,159070,159303,159516,159548,159649,159896,160252,160692,161004,161057,161290,162111,162926,163555,164190,164727,165358,165711,166191,166924,167739,168550,169448,169940,170152,170604,171246,171422,171519,171673,171857,171931,172205
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,224569,226598,228777,230940,232637,233654,236486,239129,241512,244182,246412,248070,248070,248859,251015,252577,254126,254126,255741,258543,258543,261240,261240,263172,263172,264624,264875,265716,266416,267020,267020,267551,268008,268304,268491,268940,269301,269601,269904,270164
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,223806,224383,224979,225484,226057,226749,227559,228918,230470,232325,234536,236670,238885,241406,243568,245698,247568,249310,250774,252117,253520,254885,255836,256806,257598,257976,258478,259088,259673,260191,260723,261226,261752,262165,262570,262994,263369,263685,263936,264054
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,28899,28899,29888,29888,29888,29888,29888,29888,32201,33025,33025,33025,33025,34701,35028,35028,35556,35556,35556,35958,35958,36315,36470,36599,36599,36599,36808,36808,36989,37074,37140,37140,37140,37277,37361,37452,37522,37589,37589,37589
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91907,92581,93302,93524,93694,93974,94275,94779,95220,95676,95902,96582,97263,97594,97812,97901,98029,98057,98076,98116,98226,98267,98319,98340,98351,98364,98409,98424,98453,98474,98501,98514,98514,98514,98555,98568,98585,98605,98617,98638


In [10]:
corona_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [11]:
corona_death_df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,1/12/22,1/13/22,1/14/22,1/15/22,1/16/22,1/17/22,1/18/22,1/19/22,1/20/22,1/21/22,1/22/22,1/23/22,1/24/22,1/25/22,1/26/22,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7376,7376,7376,7378,7379,7381,7383,7386,7386,7390,7390,7393,7393,7397,7401,7403,7403,7405,7408,7414,7417,7418,7420,7420,7425,7432,7442,7454,7465,7477,7478,7488,7494,7501,7504,7513,7524,7526,7534,7539
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3253,3255,3262,3265,3269,3271,3277,3283,3286,3292,3297,3305,3305,3312,3317,3323,3329,3329,3334,3346,3346,3362,3362,3371,3371,3380,3387,3393,3399,3402,3402,3407,3410,3414,3420,3423,3430,3432,3433,3438
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6372,6383,6393,6404,6412,6423,6435,6443,6453,6468,6481,6495,6508,6516,6526,6539,6545,6555,6566,6579,6593,6604,6618,6631,6640,6646,6654,6667,6678,6690,6703,6715,6729,6744,6753,6762,6772,6780,6787,6797
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,141,141,142,142,142,142,142,142,144,144,144,144,144,145,145,145,145,145,145,145,145,146,147,147,147,147,147,147,148,148,148,148,148,149,150,150,150,150,150,150
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1842,1847,1852,1857,1863,1866,1870,1877,1881,1884,1886,1888,1888,1891,1892,1893,1893,1894,1895,1895,1895,1895,1896,1896,1896,1896,1896,1896,1896,1897,1898,1898,1898,1898,1899,1899,1899,1899,1899,1899


In [12]:
corona_recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [13]:
corona_recovered_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,1/12/22,1/13/22,1/14/22,1/15/22,1/16/22,1/17/22,1/18/22,1/19/22,1/20/22,1/21/22,1/22/22,1/23/22,1/24/22,1/25/22,1/26/22,1/27/22,1/28/22,1/29/22,1/30/22,1/31/22,2/1/22,2/2/22,2/3/22,2/4/22,2/5/22,2/6/22,2/7/22,2/8/22,2/9/22,2/10/22,2/11/22,2/12/22,2/13/22,2/14/22,2/15/22,2/16/22,2/17/22,2/18/22,2/19/22,2/20/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
corona_confirmed_df = corona_confirmed_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [15]:
corona_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,variable,value
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [16]:
corona_confirmed_df = corona_confirmed_df.rename({'variable':'Date','value':'Confirmed'},axis = 'columns')

In [17]:
corona_confirmed_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [18]:
corona_death_df = corona_death_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [19]:
corona_death_df = corona_death_df.rename({'variable':'Date','value':'Death'},axis = 'columns')

In [20]:
corona_death_df.head()

,Province/State,Country/Region,Lat,Long,Date,Death
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [21]:
corona_recovered_df = corona_recovered_df.melt(id_vars=['Province/State','Country/Region','Lat','Long'])

In [22]:
corona_recovered_df = corona_recovered_df.rename({'variable':'Date','value':'Recovered'},axis = 'columns')

In [23]:
corona_recovered_df.head()

,Province/State,Country/Region,Lat,Long,Date,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0
1,NaN,Albania,41.15330,20.168300,1/22/20,0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0


In [24]:
combined_df = [corona_confirmed_df, corona_death_df, corona_recovered_df]
combined_df = [df.set_index(['Province/State','Country/Region','Lat','Long','Date']) for df in combined_df]
combined_df = combined_df[0].join(combined_df[1:])

In [25]:
combined_df.head(20)

Confirmed  ...  Recovered
Province/State               Country/Region      Lat        Long        Date                ...           
NaN                          Afghanistan          33.939110  67.709953  1/22/20        0.0  ...        0.0
                             Albania              41.153300  20.168300  1/22/20        0.0  ...        0.0
                             Algeria              28.033900  1.659600   1/22/20        0.0  ...        0.0
                             Andorra              42.506300  1.521800   1/22/20        0.0  ...        0.0
                             Angola              -11.202700  17.873900  1/22/20        0.0  ...        0.0
                             Antarctica          -71.949900  23.347000  1/22/20        0.0  ...        0.0
                             Antigua and Barbuda  17.060800 -61.796400  1/22/20        0.0  ...        0.0
                             Argentina           -38.416100 -63.616700  1/22/20        0.0  ...        0.0
                             Armenia              40.069100  45.038200  1/22/20        0.0  ...        0.0
Australian Capital Territory Australia           -35.473500  149.012400 1/22/20        0.0  ...        0.0
New South Wales              Australia           -33.868800  151.209300 1/22/20        0.0  ...        0.0
Northern Territory           Australia           -12.463400  130.845600 1/22/20        0.0  ...        0.0
Queensland                   Australia           -27.469800  153.025100 1/22/20        0.0  ...        0.0
South Australia              Australia           -34.928500  138.600700 1/22/20        0.0  ...        0.0
Tasmania                     Australia           -42.882100  147.327200 1/22/20        0.0  ...        0.0
Victoria                     Australia           -37.813600  144.963100 1/22/20        0.0  ...        0.0
Western Australia            Australia           -31.950500  115.860500 1/22/20        0.0  ...        0.0
NaN                          Austria              47.516200  14.550100  1/22/20        0.0  ...        0.0
                             Azerbaijan           40.143100  47.576900  1/22/20        0.0  ...        0.0
                             Bahamas              25.025885 -78.035889  1/22/20        0.0  ...        0.0

[20 rows x 3 columns]

In [26]:
combined_df = combined_df.reset_index()

In [27]:
combined_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Death,Recovered
0,NaN,Afghanistan,33.93911,67.709953,1/22/20,0.0,0.0,0.0
1,NaN,Albania,41.15330,20.168300,1/22/20,0.0,0.0,0.0
2,NaN,Algeria,28.03390,1.659600,1/22/20,0.0,0.0,0.0
3,NaN,Andorra,42.50630,1.521800,1/22/20,0.0,0.0,0.0
4,NaN,Angola,-11.20270,17.873900,1/22/20,0.0,0.0,0.0


In [28]:
combined_df[['Lat','Long','Confirmed','Death','Recovered']] = combined_df[['Lat','Long','Confirmed','Death','Recovered']].apply(pd.to_numeric)

In [29]:
combined_df[['Date']] = combined_df[['Date']].apply(pd.to_datetime)

In [30]:
combined_df.dtypes

Province/State            object
Country/Region            object
Lat                      float64
Long                     float64
Date              datetime64[ns]
Confirmed                float64
Death                    float64
Recovered                float64
dtype: object

In [31]:
# combined_df.Lat = combined_df.Lat.astype(int)
# combined_df.Long = combined_df.Long.astype(int)
combined_df.Confirmed = combined_df.Confirmed.astype(int)
combined_df.Death = combined_df.Death.astype(int)
combined_df.Recovered = combined_df.Recovered.fillna(0.0).astype(int)

In [32]:
combined_df.dtypes

Province/State            object
Country/Region            object
Lat                      float64
Long                     float64
Date              datetime64[ns]
Confirmed                  int64
Death                      int64
Recovered                  int64
dtype: object

In [33]:
combined_df.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Death,Recovered
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0


In [34]:
combined_df = combined_df.rename({'Province/State':'State','Country/Region':'Country'},axis = 'columns')

In [35]:
combined_df.head(5000)

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0
...,...,...,...,...,...,...,...,...
4995,NaN,Lebanon,33.854700,35.862300,2020-02-08,0,0,0
4996,NaN,Lesotho,-29.610000,28.233600,2020-02-08,0,0,0
4997,NaN,Liberia,6.428055,-9.429499,2020-02-08,0,0,0
4998,NaN,Libya,26.335100,17.228331,2020-02-08,0,0,0


In [36]:
combined_df.isnull().any()

State         True
Country      False
Lat           True
Long          True
Date         False
Confirmed    False
Death        False
Recovered    False
dtype: bool

In [37]:
combined_df.dropna(subset=['Lat', 'Long'], inplace= True)

In [38]:
combined_df.isnull().any()

State         True
Country      False
Lat          False
Long         False
Date         False
Confirmed    False
Death        False
Recovered    False
dtype: bool

In [39]:
combined_df.count()

State         66207
Country      214602
Lat          214602
Long         214602
Date         214602
Confirmed    214602
Death        214602
Recovered    214602
dtype: int64

In [40]:
combined_df.set_index('Country').filter(like="India",axis=0)

,State,Lat,Long,Date,Confirmed,Death,Recovered
Country,,,,,,,
India,NaN,20.593684,78.96288,2020-01-22,0,0,0
India,NaN,20.593684,78.96288,2020-01-23,0,0,0
India,NaN,20.593684,78.96288,2020-01-24,0,0,0
India,NaN,20.593684,78.96288,2020-01-25,0,0,0
India,NaN,20.593684,78.96288,2020-01-26,0,0,0
...,...,...,...,...,...,...,...
India,NaN,20.593684,78.96288,2022-02-16,42754315,510413,0
India,NaN,20.593684,78.96288,2022-02-17,42780235,510905,0
India,NaN,20.593684,78.96288,2022-02-18,42802505,511230,0


In [41]:
combined_df[combined_df.State.isnull()]['Country'].value_counts()

Afghanistan                         761
Saint Vincent and the Grenadines    761
New Zealand                         761
Nicaragua                           761
Niger                               761
                                   ... 
Grenada                             761
Guatemala                           761
Guinea                              761
Guinea-Bissau                       761
Zimbabwe                            761
Name: Country, Length: 195, dtype: int64

In [42]:
% pip install opencage

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 3.6 MB 14.2 MB/s 


In [43]:
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

In [44]:
geocoder = OpenCageGeocode(map_api_key)

In [45]:
results = geocoder.reverse_geocode(28.033900,	1.659600)
print(results)
print(results[0]['formatted'])
print(results[0]['formatted'].split(",")[-2].strip(" "))

[{'annotations': {'DMS': {'lat': "27° 1' 17.24844'' N", 'lng': "1° 0' 50.71572'' E"}, 'MGRS': '31RCK0298190363', 'Maidenhead': 'JL07ma15qd', 'Mercator': {'x': 112887.726, 'y': 3106745.359}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?relation=4171628#map=17/27.02146/1.01409', 'note_url': 'https://www.openstreetmap.org/note/new#map=17/27.02146/1.01409&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=27.02146&mlon=1.01409#map=17/27.02146/1.01409'}, 'UN_M49': {'regions': {'AFRICA': '002', 'DZ': '012', 'NORTHERN_AFRICA': '015', 'WORLD': '001'}, 'statistical_groupings': ['LEDC']}, 'callingcode': 213, 'currency': {'alternate_symbols': ['DA'], 'decimal_mark': '.', 'format': '%n %u', 'html_entity': '', 'iso_code': 'DZD', 'iso_numeric': '012', 'name': 'Algerian Dinar', 'smallest_denomination': 100, 'subunit': 'Centime', 'subunit_to_unit': 100, 'symbol': 'د.ج', 'symbol_first': 0, 'thousands_separator': ','}, 'flag': '🇩🇿', 'geohash': 'shb9xp47muydgn9nqs3s', 'qibla': 90.41, 'road

In [46]:
import re
english_check = re.compile(r'[a-z]')

def get_state(lati,longi):
  try:
    output = geocoder.reverse_geocode((lati,longi))[0]['formatted'].split(",")[-2].strip(" ")
    # if(len(output.split(" ")) > 1):
    #   output = "TBF"
    
    if not english_check.match(output.lower()):
      output = "TBF"

  except:
    print("Error in Reverse Geocoding")
    output = "TBF"
  return output

In [47]:
combined_df['state_cleaned'] = combined_df[combined_df.State.isnull()][['Lat','Long']].apply(lambda x: get_state(x['Lat'],x['Long']),axis=1)

Streaming output truncated to the last 5000 lines.
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Error in Reverse Geocoding
Erro

In [48]:
combined_df[combined_df.state_cleaned.isnull()]['Country'].value_counts()

China             25113
Canada            11415
United Kingdom     9893
France             8371
Australia          6088
Netherlands        3044
Denmark            1522
New Zealand         761
Name: Country, dtype: int64

In [49]:
combined_int_df = combined_df.copy()

In [50]:
combined_df[combined_df.state_cleaned.notna()]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,TBF
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,TBF
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,TBF
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,TBF
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,TBF
...,...,...,...,...,...,...,...,...,...
216119,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-20,627956,5357,0,TBF
216120,NaN,Winter Olympics 2022,39.904200,116.407400,2022-02-20,509,0,0,TBF
216121,NaN,Yemen,15.552727,48.516388,2022-02-20,11736,2122,0,TBF
216122,NaN,Zambia,-13.133897,27.849332,2022-02-20,311194,3943,0,TBF


In [51]:
combined_int_df.count()

State             66207
Country          214602
Lat              214602
Long             214602
Date             214602
Confirmed        214602
Death            214602
Recovered        214602
state_cleaned    148395
dtype: int64

In [52]:
combined_int_df[combined_int_df.State.notna()]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
9,Australian Capital Territory,Australia,-35.473500,149.012400,2020-01-22,0,0,0,NaN
10,New South Wales,Australia,-33.868800,151.209300,2020-01-22,0,0,0,NaN
11,Northern Territory,Australia,-12.463400,130.845600,2020-01-22,0,0,0,NaN
12,Queensland,Australia,-27.469800,153.025100,2020-01-22,0,0,0,NaN
13,South Australia,Australia,-34.928500,138.600700,2020-01-22,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...
216108,Isle of Man,United Kingdom,54.236100,-4.548100,2022-02-20,22297,79,0,NaN
216109,Jersey,United Kingdom,49.213800,-2.135800,2022-02-20,35301,108,0,NaN
216110,Montserrat,United Kingdom,16.742498,-62.187366,2022-02-20,163,2,0,NaN
216111,"Saint Helena, Ascension and Tristan da Cunha",United Kingdom,-7.946700,-14.355900,2022-02-20,4,0,0,NaN


In [53]:
combined_int_df[combined_int_df.State.notna()]["State"].count()

66207

In [54]:
import numpy
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: x['State'] if (numpy.all(pd.notnull(x['State']))) else x['state_cleaned'], axis=1)

In [55]:
combined_int_df[combined_int_df.State.notna()]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
9,Australian Capital Territory,Australia,-35.473500,149.012400,2020-01-22,0,0,0,Australian Capital Territory
10,New South Wales,Australia,-33.868800,151.209300,2020-01-22,0,0,0,New South Wales
11,Northern Territory,Australia,-12.463400,130.845600,2020-01-22,0,0,0,Northern Territory
12,Queensland,Australia,-27.469800,153.025100,2020-01-22,0,0,0,Queensland
13,South Australia,Australia,-34.928500,138.600700,2020-01-22,0,0,0,South Australia
...,...,...,...,...,...,...,...,...,...
216108,Isle of Man,United Kingdom,54.236100,-4.548100,2022-02-20,22297,79,0,Isle of Man
216109,Jersey,United Kingdom,49.213800,-2.135800,2022-02-20,35301,108,0,Jersey
216110,Montserrat,United Kingdom,16.742498,-62.187366,2022-02-20,163,2,0,Montserrat
216111,"Saint Helena, Ascension and Tristan da Cunha",United Kingdom,-7.946700,-14.355900,2022-02-20,4,0,0,"Saint Helena, Ascension and Tristan da Cunha"


In [56]:
combined_int_df[combined_df.state_cleaned=="TBF"]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,TBF
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,TBF
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,TBF
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,TBF
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,TBF
...,...,...,...,...,...,...,...,...,...
216119,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-20,627956,5357,0,TBF
216120,NaN,Winter Olympics 2022,39.904200,116.407400,2022-02-20,509,0,0,TBF
216121,NaN,Yemen,15.552727,48.516388,2022-02-20,11736,2122,0,TBF
216122,NaN,Zambia,-13.133897,27.849332,2022-02-20,311194,3943,0,TBF


In [57]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

Afghanistan                         761
Saint Vincent and the Grenadines    761
New Zealand                         761
Nicaragua                           761
Niger                               761
                                   ... 
Grenada                             761
Guatemala                           761
Guinea                              761
Guinea-Bissau                       761
Zimbabwe                            761
Name: Country, Length: 195, dtype: int64

In [58]:
combined_int_df.head(50)

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,TBF
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,TBF
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,TBF
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,TBF
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,TBF
5,NaN,Antarctica,-71.949900,23.347000,2020-01-22,0,0,0,TBF
6,NaN,Antigua and Barbuda,17.060800,-61.796400,2020-01-22,0,0,0,TBF
7,NaN,Argentina,-38.416100,-63.616700,2020-01-22,0,0,0,TBF
8,NaN,Armenia,40.069100,45.038200,2020-01-22,0,0,0,TBF
9,Australian Capital Territory,Australia,-35.473500,149.012400,2020-01-22,0,0,0,Australian Capital Territory


In [59]:
capital_df = pd.read_csv('https://raw.githubusercontent.com/icyrockcom/country-capitals/master/data/country-list.csv')

In [60]:
capital_df.head()

,country,capital,type
0,Abkhazia,Sukhumi,countryCapital
1,Afghanistan,Kabul,countryCapital
2,Akrotiri and Dhekelia,Episkopi Cantonment,countryCapital
3,Albania,Tirana,countryCapital
4,Algeria,Algiers,countryCapital


In [61]:
capital_df['country'].replace({"United Kingdom; England": "United Kingdom"}, inplace=True)

In [62]:
capital_df['country']=capital_df.country.str.lower()

In [63]:
capital_df.head()

,country,capital,type
0,abkhazia,Sukhumi,countryCapital
1,afghanistan,Kabul,countryCapital
2,akrotiri and dhekelia,Episkopi Cantonment,countryCapital
3,albania,Tirana,countryCapital
4,algeria,Algiers,countryCapital


In [64]:
capital_df=capital_df.set_index('country')

In [65]:
capital_df.loc['algeria'][0]

'Algiers'

In [66]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,TBF
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,TBF
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,TBF
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,TBF
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,TBF
...,...,...,...,...,...,...,...,...,...
216119,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-20,627956,5357,0,TBF
216120,NaN,Winter Olympics 2022,39.904200,116.407400,2022-02-20,509,0,0,TBF
216121,NaN,Yemen,15.552727,48.516388,2022-02-20,11736,2122,0,TBF
216122,NaN,Zambia,-13.133897,27.849332,2022-02-20,311194,3943,0,TBF


In [67]:
def lookup_capital(country):
  try:
    capital=capital_df.loc[country.lower()][0]
  except:
    capital="TBF"
  return capital  

In [68]:
combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

0                    Kabul
1                   Tirana
2                  Algiers
3         Andorra la Vella
4                   Luanda
                ...       
216119                 TBF
216120                 TBF
216121               Sanaá
216122              Lusaka
216123              Harare
Length: 214602, dtype: object

In [69]:
combined_int_df['state_cleaned']=combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [70]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

Antarctica               761
Bolivia                  761
West Bank and Gaza       761
US                       761
Timor-Leste              761
Taiwan*                  761
Summer Olympics 2020     761
Sao Tome and Principe    761
North Macedonia          761
Micronesia               761
MS Zaandam               761
Korea, South             761
Holy See                 761
Eswatini                 761
Diamond Princess         761
Czechia                  761
Cote d'Ivoire            761
Congo (Kinshasa)         761
Congo (Brazzaville)      761
Cabo Verde               761
Burma                    761
Winter Olympics 2022     761
Name: Country, dtype: int64

In [71]:
combined_int_df["Country"].replace({"Republic of Moldova": "Moldova", "Holy See": "Vatican City", "UK": "United Kingdom","Viet Nam":"Vietnam","Iran (Islamic Republic of)":"Iran"}, inplace=True)

In [72]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

Antarctica               761
Bolivia                  761
West Bank and Gaza       761
US                       761
Timor-Leste              761
Taiwan*                  761
Summer Olympics 2020     761
Sao Tome and Principe    761
North Macedonia          761
Micronesia               761
MS Zaandam               761
Korea, South             761
Vatican City             761
Eswatini                 761
Diamond Princess         761
Czechia                  761
Cote d'Ivoire            761
Congo (Kinshasa)         761
Congo (Brazzaville)      761
Cabo Verde               761
Burma                    761
Winter Olympics 2022     761
Name: Country, dtype: int64

In [73]:
combined_int_df['state_cleaned']=combined_int_df.apply(lambda x: lookup_capital(x['Country']) if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [74]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

Antarctica               761
MS Zaandam               761
West Bank and Gaza       761
US                       761
Timor-Leste              761
Taiwan*                  761
Summer Olympics 2020     761
Sao Tome and Principe    761
North Macedonia          761
Micronesia               761
Korea, South             761
Bolivia                  761
Eswatini                 761
Diamond Princess         761
Czechia                  761
Cote d'Ivoire            761
Congo (Kinshasa)         761
Congo (Brazzaville)      761
Cabo Verde               761
Burma                    761
Winter Olympics 2022     761
Name: Country, dtype: int64

In [75]:
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: x['Country'] if (x['state_cleaned']=="TBF") else x['state_cleaned'], axis=1)

In [76]:
combined_int_df[combined_int_df.state_cleaned == "TBF"]['Country'].value_counts()

Series([], Name: Country, dtype: int64)

In [77]:
combined_int_df.to_csv('corona_dataset.csv')

In [78]:
combined_int_df[combined_int_df.Country=="US"]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
256,NaN,US,40.0,-100.0,2020-01-22,1,0,0,US
540,NaN,US,40.0,-100.0,2020-01-23,1,0,0,US
824,NaN,US,40.0,-100.0,2020-01-24,2,0,0,US
1108,NaN,US,40.0,-100.0,2020-01-25,2,0,0,US
1392,NaN,US,40.0,-100.0,2020-01-26,5,0,0,US
...,...,...,...,...,...,...,...,...,...
214960,NaN,US,40.0,-100.0,2022-02-16,78174363,928639,0,US
215244,NaN,US,40.0,-100.0,2022-02-17,78275698,931862,0,US
215528,NaN,US,40.0,-100.0,2022-02-18,78423525,934321,0,US
215812,NaN,US,40.0,-100.0,2022-02-19,78457081,934951,0,US


In [79]:
usstates_df = pd.read_csv('https://raw.githubusercontent.com/jasonong/List-of-US-States/master/states.csv')

In [80]:
usstates_df.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [81]:
usstates_df=usstates_df.set_index('Abbreviation')

In [82]:
usstates_df.head()

,State
Abbreviation,
AL,Alabama
AK,Alaska
AZ,Arizona
AR,Arkansas
CA,California


In [83]:
usstates_df.loc['AZ']

State    Arizona
Name: AZ, dtype: object

In [84]:
combined_int_df

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,Kabul
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,Tirana
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,Algiers
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,Andorra la Vella
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,Luanda
...,...,...,...,...,...,...,...,...,...
216119,NaN,West Bank and Gaza,31.952200,35.233200,2022-02-20,627956,5357,0,West Bank and Gaza
216120,NaN,Winter Olympics 2022,39.904200,116.407400,2022-02-20,509,0,0,Winter Olympics 2022
216121,NaN,Yemen,15.552727,48.516388,2022-02-20,11736,2122,0,Sanaá
216122,NaN,Zambia,-13.133897,27.849332,2022-02-20,311194,3943,0,Lusaka


In [86]:
#combined_int_df['City']=combined_int_df.apply(lambda x: x['State'].split(",")[0] if(x['Country']=="US") else "", axis=1)

In [87]:
def convert_state(state):
  try:
    output=state.split(",")[1].strip()
    output=usstates_df.loc[output.upper()][0]
  except:
    output=state
  return output  

In [88]:
combined_int_df["state_cleaned"]=combined_int_df.apply(lambda x: convert_state(x['state_cleaned']) if(x['Country']=="US") else x['state_cleaned'], axis=1)

In [89]:
combined_int_df.head(50)

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
0,NaN,Afghanistan,33.939110,67.709953,2020-01-22,0,0,0,Kabul
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0,0,Tirana
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0,0,Algiers
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0,0,Andorra la Vella
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0,0,Luanda
5,NaN,Antarctica,-71.949900,23.347000,2020-01-22,0,0,0,Antarctica
6,NaN,Antigua and Barbuda,17.060800,-61.796400,2020-01-22,0,0,0,St. John's
7,NaN,Argentina,-38.416100,-63.616700,2020-01-22,0,0,0,Buenos Aires
8,NaN,Armenia,40.069100,45.038200,2020-01-22,0,0,0,Yerevan
9,Australian Capital Territory,Australia,-35.473500,149.012400,2020-01-22,0,0,0,Australian Capital Territory


In [90]:
combined_int_df[combined_int_df.Country=="US"][0:100]

,State,Country,Lat,Long,Date,Confirmed,Death,Recovered,state_cleaned
256,NaN,US,40.0,-100.0,2020-01-22,1,0,0,US
540,NaN,US,40.0,-100.0,2020-01-23,1,0,0,US
824,NaN,US,40.0,-100.0,2020-01-24,2,0,0,US
1108,NaN,US,40.0,-100.0,2020-01-25,2,0,0,US
1392,NaN,US,40.0,-100.0,2020-01-26,5,0,0,US
...,...,...,...,...,...,...,...,...,...
27236,NaN,US,40.0,-100.0,2020-04-26,970663,57850,106988,US
27520,NaN,US,40.0,-100.0,2020-04-27,995292,59330,111424,US
27804,NaN,US,40.0,-100.0,2020-04-28,1019733,61571,115936,US
28088,NaN,US,40.0,-100.0,2020-04-29,1046338,63962,120720,US


In [91]:
combined_int_df.to_csv("corona_dataset.csv")